### 자동 미분을 사용하여 신경망의 역전파 단계 연산을 자동화

 Autograd를 사용하면, 신경망의 순전파 단계에서 연산 그래프(computational graph) 를 정의하게 된다. 이 그래프의 노드(node)는 텐서(tensor)이고, 엣지(edge)는 입력 텐서로부터 출력 텐서를 만들어내는 함수. 이 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산할 수 있음.

In [1]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") -- GPU를 사용할 경우

In [2]:
# 입력과 출력값 텐서를 생성
# requires_grad = False가 기본값으로 설정되어 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가 없음을 나타냄

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

print(x.shape[0])
print(y.shape[0])

2000
2000


In [3]:
# 가중치 텐서 생성 3차 다항식이므로 4개의 가중치 필요
# requires_grad=True로 설정하여 역전파 단계 중에 이 텐서에 대한 변화도를 계산할 수 있도록 함.

a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

In [7]:
learning_rate = 1e-6
num_iteration = 5000
for t in range(num_iteration) :
  # 순전파 단계
  y_pred = a + b * x + c * x ** 2 + d * x ** 3

  # loss는 (1,) shape을 갖는 텐서입니다.
  # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
  loss = (y_pred - y).pow(2).sum()
  if t % 1000 == 999:
    print(t, loss.item())

  # autograd 를 사용하여 역전파 단계를 계산합니다. 
  # 이는 requires_grad=True를 갖는 모든 텐서들에 대한 손실의 변화도를 계산
  # 이후 a.grad와 b.grad, c.grad, d.grad는 각각 a, b, c, d에 대한 loss의 변화도를 갖는 텐서가 된다
  loss.backward()

  # 경사하강법(gradient descent)를 사용하여 가중치를 직접 갱신
  # torch.no_grad()로 감싸는 이유는, 가중치들이 requires_grad=True 지만 autograd에서 이를 추적하지 않아도 되기 때문
  with torch.no_grad():
    a -= learning_rate * a.grad
    b -= learning_rate * b.grad
    c -= learning_rate * c.grad
    d -= learning_rate * d.grad

    # 가중치 갱신 후에는 변화도를 직접 0으로 만든다 : gradient를 더하는 식으로 연산이 되기 때문
    a.grad = None
    b.grad = None
    c.grad = None
    d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

999 8.817167282104492
1999 8.817167282104492
2999 8.817167282104492
3999 8.817167282104492
4999 8.817167282104492
Result: y = 7.450554395660447e-09 + 0.8567265868186951 x + -1.1153121981521963e-08 x^2 + -0.09332836419343948 x^3


###python with문
자원을 획득하고 사용 후 반납해야 하는 경우 주로 사용

1) 자원을 획득한다

2) 자원을 사용한다

3) 자원을 반납한다

​